#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
              
        for line in csvreader:
            full_data_rows_list.append(line) 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length', 'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


#### Number of rows in csv file

In [5]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Begin writing your Apache Cassandra code in the cells below

#### Creating a local cluster

In [6]:
# Local connection to Cassandra 
host = '127.0.0.1'

from cassandra.cluster import Cluster
cluster = Cluster([host])

try:
    session = cluster.connect()
except Exception as e:
    print(e)

#### Creating Keyspace

In [7]:
create_ksp = """

    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = {
        'class': 'SimpleStrategy', 'replication_factor': 1
    };
    
"""
try:
    session.execute(create_ksp)
except Exception as e:
    print(e)

#### Setting Keyspace

In [8]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### First demand: Create table to return artist, song title and song's length in the music app history that was heard during ```sessionId = 338 and itemInSession  = 4```

#### Choosing ```session_id``` and ```item_in_session``` as primary key and partition key, since we are going to read data based on these columns and because we can distribute data in more number of nodes, improving query performance and avoiding node unbalance.

In [12]:
query_drop_session_songs = "DROP TABLE IF EXISTS session_songs;"

query_create_session_songs = """

    CREATE TABLE IF NOT EXISTS session_songs (
        session_id int, 
        item_in_session int, 
        artist text, 
        song_title text, 
        length float,
        PRIMARY KEY((session_id, item_in_session))
    );
    
"""

try:
    session.execute(query_drop_session_songs)
    session.execute(query_create_session_songs)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

In [13]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (session_id, item_in_session, artist, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

#### Verifying that the data have been inserted into ```session_songs``` and testing query conditions

In [14]:
query1 = """

    SELECT 
          artist
        , song_title
        , length
    FROM 
        session_songs 
    WHERE 
        session_id = 338 
        AND item_in_session = 4;
        
"""

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.session_id, row.item_in_session, row.artist, row.song_title, row.length)

338 4 Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Second demand: Create a table to return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for ```userid = 10 and sessionid = 182```

#### Choosing ```user_id``` and ```session_id``` as partition key to optimize query demand and item_in_session as clustering column to sort values in nodes (in practice, it will be faster to find for recent sessions itens)

In [16]:
query_drop_user_session_songs = "DROP TABLE IF EXISTS user_session_songs;"

query_create_user_session_songs = """

    CREATE TABLE IF NOT EXISTS user_session_songs (
        user_id int,
        session_id int,
        item_in_session int,
        artist text, 
        song_title text, 
        user_first_name text, 
        user_last_name text, 
        PRIMARY KEY((user_id, session_id), item_in_session)
    )
        WITH CLUSTERING ORDER BY (item_in_session DESC);
        
"""

try:
    session.execute(query_drop_user_session_songs)
    session.execute(query_create_user_session_songs)
except Exception as e:
    print(e)                    

In [17]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_session_songs (user_id, session_id, item_in_session, artist, song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[0]), str(line[9]), str(line[1]), str(line[4])))

#### Verifying that the data have been inserted into ```user_session_songs``` and testing query conditions

In [20]:
query2 = """

    SELECT 
          artist
        , song_title
        , user_first_name
        , user_last_name 
    FROM 
         user_session_songs 
    WHERE 
        user_id = 10 
        AND session_id = 182;
        
"""

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.user_first_name, row.user_last_name)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz


### Third demand: Create a table to return every user name (first and last) in music app history who listened to the ```song = 'All Hands Against His Own'```

#### Choosing ```song_tile``` as partition key to find user data by songs faster. ```song_title``` and ```user_id``` as primary key allow us to have uniqueness values in the table

In [21]:
query_drop_user_songs = """DROP TABLE IF EXISTS user_songs;"""

query_create_users_songs = """

    CREATE TABLE IF NOT EXISTS user_songs (
        song_title text,
        user_id int,
        user_first_name text,
        user_last_name text,
        PRIMARY KEY ((song_title), user_id)
    );
    
"""

try:
    session.execute(query_drop_user_songs)
    session.execute(query_create_users_songs)
except Exception as e:
    print(e)

In [23]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s);"
        session.execute(query, (str(line[9]), int(line[10]), str(line[1]), str(line[4])))

#### Verifying that the data have been inserted into ```user_songs``` and testing query conditions

In [24]:
query3 = """

    SELECT 
          user_first_name
        , user_last_name 
    FROM 
        user_songs 
    WHERE 
        song_title = 'All Hands Against His Own';
        
"""

try:
     rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print(row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping tables before closing out the sessions

In [25]:
try:
    session.execute(query_drop_session_songs)
    session.execute(query_drop_user_session_songs)
    session.execute(query_drop_user_songs)
except Exception as e:
    print(e)

### Closing session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()